<span style="font-size:36px"><b>Model Inference</b></span>

Copyright &copy; 2020 Gunawan Lumban Gaol

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language overning permissions and limitations under the License.

# Import Packages

In [ ]:
import os
import sys
import glob
import warnings

import yaml
import numpy as np
import IPython.display as ipd
import librosa.display as ld
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from tqdm.auto import tqdm

from gurih.data.splitter import Splitter
from gurih.data.normalizer import AudioNormalizer
from gurih.features.extractor import MFCCFeatureExtractor
from gurih.models.model import BaselineASRModel
from gurih.models.decoder import CTCDecoder
from gurih.models.utils import CharMap

%load_ext autoreload
%autoreload 2

In [ ]:
def init(Pipeline, model_idx=None):
    """
    Initialize model in Pipeline.
    
    Parameters
    ----------
    Pipeline : Pipeline
        correcty configured pipeline
    model_idx : int
        number indicating model step in pipeline
    
    Returns
    -------
    Pipeline : Pipeline
        initialized pipeline
    """
    if model_idx is None:
        raise ValueError('Please provide model_idx param indicating model steps in pipeline.')
    
    model = Pipeline[model_idx]
    model.compile()
    model.load(model.dir_path)
    
    return Pipeline

In [ ]:
def clean_mfcc(x_freq):
    """
    For visualization, remove frequency array with zero values
    on 2nd to last dimension of mfcc because earlier zero pad
    
    Parameters
    ----------
    x_freq : numpy.ndarray[shape=(frames, n_mfcc)]
        mfcc sequence
        
    Returns
    -------
    x_freq : list [shape=(:idx_remove, n_mfcc)]
        clean mfcc sequence
    """
    check = np.abs(np.round(x_freq))
    idx_remove = None
    for i in range(check.shape[0], -1, -1):
        stop = check[i:, 1:].any()
        if stop:
            idx_remove = i+1
            return x_freq[:idx_remove]

In [ ]:
def infer(X, Pipeline, sep='\n', debug=False, plot=False, progressbar=True):
    """
    Create audio transcript for given audio filename.
    
    Parameters
    ----------
    X : numpy.ndarray[shape=(m, )]
        numpy array of audio filenames
    Pipeline : Pipeline
        initialized pipeline
    sep : str
        separator string for prediction output chunk
    debug : bool, [default=False]
        if True, print output shape of each pipeline process
    plot : bool, [default=False]
        if True, will plot normalized audio waveform and frequency
        domain features spectrum. Note that by enabling this, the
        pipeline will require more memory to store variables for
        plotting.
    progressbar : bool, [default=False]
        if True, will draw progress bar on inference process

    Returns
    -------
    y_preds : numpy.ndarray[shape=(m, c)]
        audio transcript of given file input, with chunks denoted
        by sep parameters
    """
    # Preprocessing Stage
    X = Pipeline[0].fit_transform(X)
    
    y_preds = []
    if progressbar is True: X = tqdm(X)
    for x in X:
        if x.shape[0] != 1 and Pipeline[0].mono is False:
            warnings.warn("Performing channel split and transcripting"
                          " for each channel. You can force mono"
                          " process by passsing mono=True on the"
                          " first pipeline step.")
        y_pred_c = []
        for x_c in x:
            x_c = np.expand_dims(x_c, axis=0)
            if plot is True: x_c_cache = x_c.copy()  # store normalized waveform
            if debug is True: print(x_c.shape)
            
            # Preprocessing
            for pipe in Pipeline[1:3]:
                x_c = pipe.fit_transform(x_c)
                if debug is True: print(x_c.shape)
            if plot is True: x_freq_cache = x_c.copy()  # store frequency features

            # Processing (Model)
            x_c = Pipeline[3].predict(x_c)
            if debug is True: print(x_c.shape)
                
            # Post-processing
            y_pred = Pipeline[4].fit_predict(x_c)
            y_pred_str = sep.join([y for y in y_pred])
            
            if plot is True:
                # Create figure for visualization
                plt.figure(figsize=(15, 6))
                plt.subplot(2, 1, 1)
                _ = ld.waveplot(np.asfortranarray(x_c_cache)[0], sr=sr)
                plt.title("Normalized Audio")
                plt.subplot(2, 1, 2)
                _ = ld.specshow(clean_mfcc(x_freq_cache.reshape(-1, x_freq_cache.shape[-1])).T, sr=sr, y_axis='log')
                plt.title("MFCC Features Audio")
                plt.tight_layout()
                plt.show()
            if debug is True: print(y_pred_str+'\n')
            
            y_pred_c.append(y_pred_str)
            
        y_preds.append(y_pred_c)
        
    return y_preds

# Inference Pipeline

## Create with Public API

Create your own pipeline using available public API from `gurih` packages.

In [ ]:
# Define base parameters for your pipeline.
sr = 16000
force_mono = False

Create the pipeline:
1. The minimum pipeline should be `AudioNormalizer`, `Splitter`, and `gurih.models.model` instance.
2. Depending on the model, you could add extra decoder part to get string output.
3. This minimum pipeline will just use the single features (amplitude) from signals as input of the model.

In [ ]:
pipeline = [
    AudioNormalizer(sample_rate=sr,
                    mono=force_mono,
                    write_audio_output=False,  # don't output normalized audio
                    output_dir="",
                    encode=False),  # don't output .json
    Splitter(max_frame_length=16000*30+81,  # sample-rate * (sr_norm / sr_mfcc * stride_mfcc) + (sr_mfcc * stride_mfcc / 2 + 1)
             strides=16000*30+81,
             padding='same',
             low_memory=False),
    MFCCFeatureExtractor(sample_rate=sr,
                         frame_size=0.025,
                         frame_stride=0.01,
                         filter_num=26,
                         cep_num=13,
                         NFFT=512,
                         low_freq=0,
                         high_freq=None,
                         pre_emphasis_coeff=0.97,
                         cep_lifter=22,
                         dct_type=2,
                         dct_norm="ortho",
                         append_energy=True,
                         append_delta=True,
                         low_memory=False,
                         write_output=False,
                         output_dir="."),
    BaselineASRModel(input_shape=(3000, 39), 
                     vocab_len=29, 
                     training=False,  # pass false to not save model config
                     dir_path="../../models/Model010b/BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.h5",
                     doc_path="../../docs/Model010b/"),
    CTCDecoder(CharMap.IDX_TO_CHAR_MAP)
]

In [ ]:
# Save created pipeline
config_filename = "../../docs/Model010b/config.yaml"
with open(config_filename, 'w') as f:
    f.writelines(yaml.dump(pipeline))

## Load Saved Pipeline

Optionally, you can also load your `.yaml` preconfigured pipeline file.

In [ ]:
config_filename = "../../docs/Model010b/config.yaml"
with open(config_filename, 'r') as f:
    pipeline = yaml.full_load(f)

In [ ]:
pipeline

## Initialize Pipeline

First, prepare the model by compiling and loading trained model.

In [ ]:
model_idx = 3
pipeline = init(pipeline, model_idx=model_idx)

## Create Audio Transcript

Use the pipeline to create audio transcript. You just simply use `infer` method which accepts a `Pipeline` object and a saved model path.

In [ ]:
# Multiple files from a directory
input_dir = "../../dataset/sample/"
mp3_files = glob.glob(input_dir+"*.mp3")
X = mp3_files
print(f"Total audio files: {len(X)}")

In [ ]:
X

Create transcription from input audio by following steps:
1. If channels > 1, split audio files into each channel
2. For each channel, if audio `frames > max_seq_length` then the audio will be splitted before transcription

In [ ]:
y_preds = infer(X, pipeline, sep='\n', progressbar=True)
y_preds.shape

# Inference Pipeline with Plot

Inference pipeline with plot, great for single file usage to inspect more details.

In [ ]:
# Single file
mp3_file = "../../dataset/sample/202102_1.mp3"
x = np.array([mp3_file])

In [ ]:
x

Create transcription from input audio by following steps:
1. If channels > 1, split audio files into each channel
2. For each channel, if audio `frames > max_seq_length` then the audio will be splitted before transcription

In [ ]:
y_pred = infer(x, pipeline, sep='\n', plot=True, progressbar=False)
y_pred.shape

Run below cell to hear what the audio is like.

In [ ]:
ipd.Audio(x[0])

You can then inspect the output prediction from the pipeline.

In [ ]:
y_pred